### Imports

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from scipy.stats import norm
# from scipy.stats import bootstrap
import math

### Read every cohort study file

In [2]:
datasets = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../cohort_studies_full_data/' + "/*."+'csv'))]
cohorts = [file.split(".")[0] for file in sorted(os.listdir('../cohort_studies_full_data/'))]

In [3]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies = dict()
# dfsss = dict()

for cohort, dataset in zip(cohorts, datasets):
    cohort_n = cohort.split("_MERGE")[0]
    cohort_studies[cohort_n] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit
#     dfsss[cohort_n] = dataset

In [4]:
datasets_sub = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../preprocessed_datasets/' + "/*."+'csv'))]
cohorts_sub = [file.split(".")[0] for file in sorted(os.listdir('../preprocessed_datasets/'))]

In [5]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies_sub = dict()

for cohort, dataset in zip(cohorts_sub, datasets_sub):
    cohort_studies_sub[cohort] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit

In [6]:
for i in cohort_studies:
    cohort_studies[i]['Age']=cohort_studies_sub[i]['Age']

### Read harmonized mapping tables

In [7]:
modality = [pd.read_csv(file, sep=',') for file in sorted(glob.glob('../feature_tables' + "/*."+'csv'))]
name = [file.split(".")[0] for file in sorted(os.listdir('../feature_tables'))]

In [8]:
# make a dictionary that contains all modalities as a dataframe
mappings = dict()

for moda, na in zip(modality, name):
    mappings[na.split(' - ')[1]] = moda

In [9]:
harmonized_features = pd.concat(mappings, ignore_index=True) # combine all tables

In [10]:
# exclude categorical and taboo features
harmonized_features = harmonized_features.loc[(harmonized_features['Rank']!=1) & (harmonized_features['Rank']!=2)]

### Read the feature availability files for all cohorts

In [11]:
ava_mapp = [pd.read_csv(file, sep='\t') for file in sorted(glob.glob('../feature_availability_in_cohorts' + "/*."+'tsv'))]
tablesss = [file.split(".")[0] for file in sorted(os.listdir('../feature_availability_in_cohorts'))]

In [12]:
# make a dictionary that contains all modalities as a dataframe
available_features = dict()

for modal, df in zip(tablesss, ava_mapp):
    available_features[modal] = df

In [13]:
existing_features = pd.concat(available_features, ignore_index=True) # combine all tables

In [14]:
existing_features.replace({0: np.nan}, inplace=True) # 0 indicates that the feature was not measured 

### Selecetion of cohort studies for A/T/N assignment

### Select the patient that have CSF biomarker, disregard the diagnostic status

In [15]:
atn = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=mappings['csf'].Feature.loc[0:2].to_list()+(["Total"]))
# atn = pd.DataFrame(index=cohort_studies, columns=['A', 'T', 'N'])

In [16]:
for cohort in atn.index:
    for feat in mappings['csf'][cohort].loc[0:2].dropna().to_list():
        if feat in cohort_studies[cohort].columns:
            atn.loc[cohort, mappings['csf'].loc[mappings['csf'][cohort]==feat, 'Feature']] = len(cohort_studies[cohort][feat].dropna())
            atn.loc[cohort, 'Total'] = len(cohort_studies[cohort][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

In [17]:
# atn

In [18]:
diag = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=cohort_studies['ADNI']['Diagnosis'].dropna().unique())

In [19]:
for cohort in diag.index:
    for dia in diag.columns:
        diag.loc[cohort, dia] = len(cohort_studies[cohort].loc[cohort_studies[cohort]['Diagnosis']==dia][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

In [20]:
# diag

### Remove the empty columns from all cohorts that we are intrested in
### Remove the participant without all 3 CSF biomarkers

In [21]:
selected_cohorts = dict()

for coh in diag.index:
    selected_cohorts[coh] = cohort_studies[coh].dropna(axis=1, how='all')

In [22]:
total_feats = dict()

# existing_features.set_index('Feature', inplace=True)

for feat in existing_features.Feature:
    total_feats[feat] = existing_features.loc[existing_features.Feature==feat][selected_cohorts].dropna(axis=1).columns

In [23]:
for cohort in atn.index:
    feat = mappings['csf'][cohort].loc[0:2].dropna().to_list()
    cohort_studies[cohort] = cohort_studies[cohort].dropna(subset=feat)

As Some features have suffix due to merging tables for certain cohorts, first investigate if all the harmonized features are in cohorts. Rename the ones that have suffix so it can be compatible to work with our harmonized names.

In [24]:
cohort_studies['ADNI'].rename(columns={'PTEDUCAT_x': 'PTEDUCAT', 'TRABSCOR_bl': 'TRABSCOR'}, inplace=True)

### Clustering CSF biomarkers, two classes, normal vs abnormal

### subset each cohort dataset based on the columns of interest for clustering 

In [25]:
cohorts_csf = dict()

for i in atn.index:
    csf = mappings['csf'].iloc[:3][i].to_list()
    
    if i == 'NACC':
        cohorts_csf['NACC_ELISA'] = cohort_studies[i].loc[cohort_studies[i]['CSFTTMD']==1][csf + ["Diagnosis", "Age"]] # ELISA
        cohorts_csf['NACC_XMAP'] = cohort_studies[i].loc[cohort_studies[i]['CSFTTMD']==2][csf + ["Diagnosis", "Age"]] # xmap
        cohorts_csf['NACC_ELISA'] = cohorts_csf['NACC_ELISA'].dropna(subset=cohorts_csf['NACC_ELISA'].columns[:3].to_list() + ['Age']) # drop empty rows (CSF biomarkers)
        cohorts_csf['NACC_XMAP'] = cohorts_csf['NACC_XMAP'].dropna(subset=cohorts_csf['NACC_XMAP'].columns[:3].to_list() + ['Age']) # drop empty rows (CSF biomarkers)

    
    elif i == 'EMIF':
        cohorts_csf['EMIF_ELISA'] = cohort_studies[i].loc[~(cohort_studies[i]['Studyname'].isin(['EDAR', 'Leuven', ]))][csf + ["Diagnosis", "Age"]] # INNOTEST ELISA
        cohorts_csf['EMIF_XMAP'] = cohort_studies[i].loc[(cohort_studies[i]['Studyname'].isin(['EDAR', 'Leuven', ]))][csf + ["Diagnosis", "Age"]] # xmap and not collected
        cohorts_csf['EMIF_ELISA'] = cohorts_csf['EMIF_ELISA'].dropna(subset=cohorts_csf['EMIF_ELISA'].columns[:3].to_list() + ['Age']) # drop empty rows (CSF biomarkers)
        cohorts_csf['EMIF_XMAP'] = cohorts_csf['EMIF_XMAP'].dropna(subset=cohorts_csf['EMIF_XMAP'].columns[:3].to_list() + ['Age']) # drop empty rows (CSF biomarkers)

    else: 
        cohorts_csf[i] = cohort_studies[i][csf + ["Diagnosis", "Age"]]
        cohorts_csf[i] = cohorts_csf[i].dropna(subset=cohorts_csf[i].columns[:3].to_list() + ['Age']) # drop empty rows (CSF biomarkers)

In [26]:
cohorts_csf['ADNI'].sample(n=len(cohorts_csf['ADNI'].index), replace=True)

,PTAU,TAU,ABETA,Diagnosis,Age
RID,,,,,
5194,11.81,119.7,1386.0,CU,65.2
394,33.87,321.3,643.4,MCI,84.0
139,45.14,395.9,651.9,AD,65.9
855,58.05,495.7,638.3,MCI,75.6
4337,9.90,114.2,998.6,CU,72.0
...,...,...,...,...,...
4312,20.75,248.8,666.2,MCI,69.3
1062,15.51,179.7,424.7,AD,81.9
4712,51.87,513.7,1127.0,MCI,74.3


# Bootstrap, scale features, train model, extract cutoffs and categorize the participants

In [27]:
def reverse_standardization(dfs, cccs, csf_mappings):
    """
    dfs: a dictionary of dataframes where each df containes all CSF measurements, scaled CSF, etc. 
    cccs: df containing each cluster mean and SD for all CSF biomarkers
    csf_mappings: harmonized mapping of CSF biomarkers among the investigated cohorts
    """
    
    for study in dfs:
        
        if '_' not in study:
        
            for biomarker in csf_mappings['csf'].iloc[:3][study].to_list():
                min_, max_ = np.min(dfs[study][biomarker]), np.max(dfs[study][biomarker])
                featu = csf_mappings['csf'].loc[csf_mappings['csf'][study]==biomarker, 'Feature'].item()

                for prefix in ['mean1_', 'mean2_', 'var1_', 'var2_']:
                    cccs.loc[study, prefix + featu + "_rs"]= round(cccs.loc[study][prefix + featu] * (max_ - min_) + min_, 2)

        else:
            study_n = study.split("_")[0]
            
            for biomarker in csf_mappings['csf'].iloc[:3][study_n].to_list():
                min_, max_ = np.min(dfs[study][biomarker]), np.max(dfs[study][biomarker])
                featu = csf_mappings['csf'].loc[csf_mappings['csf'][study_n]==biomarker, 'Feature'].item()

                for prefix in ['mean1_', 'mean2_', 'var1_', 'var2_']:
                    cccs.loc[study, prefix + featu + "_rs"]= round(cccs.loc[study][prefix + featu] * (max_ - min_) + min_, 2)

In [28]:
def solve(m1, m2, std1, std2):
    a = 1/(2*std1**2) - 1/(2*std2**2)
    b = m2/(std2**2) - m1/(std1**2)
    c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
    return np.roots([a,b,c])

In [29]:
def cutoffs_from_bootstrap_dfs(all_cohorts, iteration_):
    
    for boot in range(iteration_):
        bootstraped_dfs = dict()
        my_cols = list()
        bio_list = list()
        
        
        # Sample from the data with replacement
        for study in all_cohorts:
            bootstraped_dfs[study] = all_cohorts[study].sample(n=len(all_cohorts[study].index), replace=True)
            bootstraped_dfs[study].to_csv(f"../results/bootstrap/gmm/datasets/{study}_{boot}.csv") #save sampled dfs
    
        # scale each feature and write it as a new column in the respective dataframe
        # name the new columns with a suffix "_scaled"
        for cohort in bootstraped_dfs:

            for i in bootstraped_dfs[cohort][bootstraped_dfs[cohort].columns[:3].to_list() + ['Age']]: 
                bootstraped_dfs[cohort][i + '_scaled'] = MinMaxScaler().fit_transform(X=bootstraped_dfs[cohort][[i]])
                
                
        # add a prefix to each biomarker, for mean and variace obtained from each distribution
        for prefix in ['mean1_', 'mean2_', 'var1_', 'var2_']:

            for i in mappings['csf']['Feature'][:3].to_list():
                my_cols.append(prefix + i)

        # make an empty df to save the mean and covariances        
        mean_variances_df = pd.DataFrame(index=bootstraped_dfs.keys(), columns=my_cols)
        
        
        # Train GMM with two clusters 
        for cname in bootstraped_dfs:
            
            if '_' not in cname:

                for col in bootstraped_dfs[cname].columns[:3]:
                    variable = mappings['csf'].loc[mappings['csf'][cname]==col, 'Feature'].item()
                    # Set the model and its parameters
                    model = GaussianMixture(n_components=2, n_init=20)
                    # Fit the model 
                    clm = model.fit(bootstraped_dfs[cname][[col + '_scaled', 'Age_scaled']])
                    # extract the mean and std of each Gaussian
                    mean_variances_df.loc[cname, 'mean1_' + variable] = clm.means_.tolist()[0][0]
                    mean_variances_df.loc[cname, 'mean2_' + variable] = clm.means_.tolist()[1][0]
                    # note: we can extract the covariance matrix where the diagonal elements are variance
                    # square root of variance will result in std
                    mean_variances_df.loc[cname, 'var1_' + variable] = math.sqrt(clm.covariances_.tolist()[0][0][0])
                    mean_variances_df.loc[cname, 'var2_' + variable] = math.sqrt(clm.covariances_.tolist()[1][0][0])
                    bootstraped_dfs[cname][col + "_ATN"] = clm.predict(bootstraped_dfs[cname][[col + '_scaled', 'Age_scaled']])


            else:
                
                cname_new = cname.split("_")[0]

                for col in bootstraped_dfs[cname].columns[:3]:
                    variable = mappings['csf'].loc[mappings['csf'][cname_new]==col, 'Feature'].item()
                    # Set the model and its parameters
                    model = GaussianMixture(n_components=2, n_init=20)
                    # Fit the model 
                    clm = model.fit(bootstraped_dfs[cname][[col + '_scaled', 'Age_scaled']])
                    # extract the mean and std of each Gaussian
                    mean_variances_df.loc[cname, 'mean1_' + variable] = clm.means_.tolist()[0][0]
                    mean_variances_df.loc[cname, 'mean2_' + variable] = clm.means_.tolist()[1][0]
                    # note: we can extract the covariance matrix where the diagonal elements are variance
                    # square root of variance will result in std
                    mean_variances_df.loc[cname, 'var1_' + variable] = math.sqrt(clm.covariances_.tolist()[0][0][0])
                    mean_variances_df.loc[cname, 'var2_' + variable] = math.sqrt(clm.covariances_.tolist()[1][0][0])
                    bootstraped_dfs[cname][col + "_ATN"] = clm.predict(bootstraped_dfs[cname][[col + '_scaled', 'Age_scaled']])

                
        reverse_standardization(bootstraped_dfs, mean_variances_df, mappings)
        mean_sd_df = mean_variances_df[[col for col in mean_variances_df.columns if '_rs' in col]]
        cutpointssss_GMM = pd.DataFrame(index=bootstraped_dfs.keys(), columns=mappings['csf'].iloc[:3]['Feature'].to_list())
        
        for ind in bootstraped_dfs:

            if "_" not in ind:
                csf = mappings['csf'].iloc[:3][ind].to_list()
                csf_harmon = mappings['csf'].iloc[:3]['Feature'].to_list()

                for bioma, bioma_har in zip(csf, csf_harmon):
                    m11, std11 = mean_sd_df.loc[ind][['mean1_'+ bioma_har + '_rs', 'var1_'+ bioma_har + '_rs']]
                    m22, std22 = mean_sd_df.loc[ind][['mean2_'+ bioma_har + '_rs', 'var2_'+ bioma_har + '_rs']]
                    result = solve(m11, m22, std11, std22)
                    cutpointssss_GMM.loc[ind, mappings['csf'].loc[mappings['csf'][ind]==bioma, 'Feature']] = round(result[np.argmax(norm.pdf(result, m11, std11))], 2)

            else:
                csf = mappings['csf'].iloc[:3][ind.split('_')[0]].to_list()
                csf_harmon = mappings['csf'].iloc[:3]['Feature'].to_list()

                for bioma, bioma_har in zip(csf, csf_harmon):
                    m11, std11 = mean_sd_df.loc[ind][['mean1_'+ bioma_har + '_rs', 'var1_'+ bioma_har + '_rs']]
                    m22, std22 = mean_sd_df.loc[ind][['mean2_'+ bioma_har + '_rs', 'var2_'+ bioma_har + '_rs']]
                    result = solve(m11, m22, std11, std22)
                    cutpointssss_GMM.loc[ind, mappings['csf'].loc[mappings['csf'][ind.split('_')[0]]==bioma, 'Feature']] = round(result[np.argmax(norm.pdf(result, m11, std11))], 2)


        
        # Rearrange the columns to A T N 
        cutpointssss_GMM = cutpointssss_GMM[['A-beta 1-42 in CSF', 'pTau in CSF', 'tTau in CSF']]
        cutpointssss_GMM = cutpointssss_GMM.loc[['ADNI', 'EPAD', 'AIBL', 'ARWIBO', 'EDSD', 'PREVENT-AD', 'PharmaCog', 'NACC_ELISA', 'EMIF_ELISA', 'NACC_XMAP', 'EMIF_XMAP', 'DOD-ADNI', 'JADNI']]
        cutpointssss_GMM.to_csv(f"../results/bootstrap/gmm/cutoffs/gmm_cutoffs_{boot}.csv") # save cutoffs
        
        
        for a in ['A', 'T', 'N']: 
            for b in ['+', '-']:
                bio_list.append(a+b)
        
        profiles_df = pd.DataFrame(index=cutpointssss_GMM.index, columns=bio_list)
        classes = {i: pd.DataFrame(index=all_cohorts[i].index, columns=['A', 'T', 'N']) for i in cutpointssss_GMM.index}
        for i in classes: classes[i].replace({np.nan: 0}, inplace=True)
            
            
        for ind in cutpointssss_GMM.index:
    
            if "_" not in ind:

                for col, letter in zip(cutpointssss_GMM.columns, ['A', 'T', 'N']):
                    threshold = cutpointssss_GMM.loc[ind][col]
                    bio = mappings['csf'].loc[mappings['csf']['Feature']==col, ind].item()

                    if letter == 'T':
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]>threshold].index, "T"] = 1
                    
                    elif letter == 'N':
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]>threshold].index, "N"] = 1
                    
                    else:
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]<threshold].index, "A"] = 1


            else:

                 for col, letter in zip(cutpointssss_GMM.columns, ['A', 'T', 'N']):
                    threshold = cutpointssss_GMM.loc[ind][col]
                    bio = mappings['csf'].loc[mappings['csf']['Feature']==col, ind.split("_")[0]].item()

                    if letter == 'T': 
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]>threshold].index, "T"] = 1
                    
                    elif letter == 'N':
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]>threshold].index, "N"] = 1
                    
                    else:
                        classes[ind].loc[all_cohorts[ind].loc[all_cohorts[ind][bio]<threshold].index, "A"] = 1
                        
        for i in classes:
            classes[i]['ATN'] = classes[i]['A'].astype(str) + classes[i]['T'].astype(str) + classes[i]['N'].astype(str)

        final_profiles = pd.DataFrame(index=classes, columns=list(Counter(classes['ADNI']['ATN']).keys()))
        final_profiles.replace({np.nan: 0}, inplace=True)

        for i in classes:
            profs = dict(Counter(classes[i]['ATN']))

            for pro in profs:
                final_profiles.loc[i, pro] = profs[pro]
        
        final_profiles.rename(columns={'000': "A-T-N-", '100': 'A+T-N-', '111': 'A+T+N+', '110': 'A+T+N-', 
                               '011': "A-T+N+", '101': "A+T-N+", '001': 'A-T-N+', '010': 'A-T+N-'}, inplace=True)
        final_profiles = final_profiles[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A-T+N-', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]
        final_profiles.replace({np.nan: 0}, inplace=True)
        final_profiles.loc['NACC'] = final_profiles.loc['NACC_ELISA'] + final_profiles.loc['NACC_XMAP']
        final_profiles.loc['EMIF'] = final_profiles.loc['EMIF_ELISA'] + final_profiles.loc['EMIF_XMAP'] 
        final_profiles = final_profiles.loc[['ADNI', 'EPAD', 'AIBL', 'ARWIBO', 'EDSD', 'PREVENT-AD', 'PharmaCog', 'NACC', 'EMIF', 'DOD-ADNI', 'JADNI']]
        final_profiles.to_csv(f"../results/bootstrap/gmm/profiles/gmm_profiles_{boot}.csv")

In [33]:
cutoffs_from_bootstrap_dfs(cohorts_csf, iteration_=1000)